In [42]:
import yaml
import torch
%load_ext autoreload
%autoreload 2
from aircraft_env import Location
from aircraft_env import AircraftEnv
from utils.units_converter import feet2meter

from itertools import product
from ddqn_agent.dqn_agent import DQNAgent
from ddqn_agent.ddqn_agent import DDQNAgent
from ddqn_agent.ddqnF_agent import DDQNFAgent
from double_dqn.double_dqn_agent import DoubleDQNAgent
import numpy as np
with open('ddqn_agent/config.yaml') as f:
    templates = yaml.safe_load(f)

# with open('double_dqn/config.yaml') as f:
#     templates = yaml.safe_load(f)
loc_1 = Location(45.46873715, -73.74257166095532)
loc_2 = Location(49.0068908, 2.5710819691019156)
loc_1, loc_2
env = AircraftEnv(arrival_location=loc_1, destination=loc_2, n_waypoints=9)
agent = DQNAgent(templates, env=env)
# agent = DDQNFAgent(templates, env)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Actions Number 150


In [34]:
agent.train()

Episode: 24550 | Epsilon: 0.3995592579054319:  49%|████▉     | 24548/50000 [02:30<02:31, 168.44it/s] 

-9634.949598942716
-9625.843132744465
-9485.640828740343


Episode: 24900 | Epsilon: 0.3941434275654093:  50%|████▉     | 24930/50000 [02:32<02:26, 171.11it/s] 

-9382.259328567066


Episode: 25800 | Epsilon: 0.38056022870630923:  52%|█████▏    | 25844/50000 [02:38<02:12, 182.51it/s]

...

Episode: 26350 | Epsilon: 0.3724969252302901:  53%|█████▎    | 26364/50000 [02:40<02:10, 180.79it/s] 

-9175.490395333813


Episode: 33750 | Epsilon: 0.27962065568226085:  68%|██████▊   | 33778/50000 [03:20<01:21, 198.70it/s]

-9079.25979491396


Episode: 49950 | Epsilon: 0.1510361149721288: 100%|██████████| 50000/50000 [04:40<00:00, 178.39it/s] 


In [43]:
from utils.env_extra import env_summary
# templates['load_model'] = True
# templates['path_to_load'] = templates['path_to_save'] + 'best'
# new_agent = DDQNAgent(templates, env)
df = env_summary(env, agent=agent, random=False)
df

,trajectory,waypoint,speed,tas,distance,wind_magnitude,wind_direction,course,correction_angle,altitude,time_for_distance,fuel_burn,total_time,reward,fuel_flow,mach_number
0,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,0.00
1,1,1,230.621472,226.896182,694412.346818,3.930965,71.027558,52.567879,-0.314310,10209.6,3011.048111,1177.256169,0.000000,822.743831,0.39097886380312863,0.76
2,1,2,222.847469,224.860213,692673.413059,2.135966,261.850609,62.202200,0.183005,10819.2,3108.284854,1153.931762,3011.048111,846.068238,0.371243890525617,0.76
3,1,3,220.858655,224.252815,692673.930453,13.545263,326.163992,68.981095,3.376488,11428.8,3136.277048,1108.090575,6119.332965,891.909425,0.353313995610684,0.76
4,1,4,219.885365,224.252815,692674.248907,38.774459,168.151850,76.685708,-9.953479,12038.4,3150.160767,1063.357843,9255.610013,936.642157,0.3375566903337691,0.76
5,2,5,200.970213,227.203510,694483.019994,43.721879,211.240506,89.208892,-9.389069,12038.4,3455.651517,1186.735286,12405.770780,813.264714,0.34341868105128626,0.77
6,2,6,201.430390,227.203510,692678.387980,26.444102,287.467590,93.716501,1.585364,12038.4,3438.797822,1180.947412,15861.422297,819.052588,0.34341868105128626,0.77
7,2,7,222.071661,227.203510,692678.387980,8.700504,227.323299,102.079322,-1.792200,12038.4,3119.166059,1071.179894,19300.220119,928.820106,0.34341868105128626,0.77
8,2,8,244.580755,227.203510,692678.387980,18.291803,127.339210,109.825991,-1.388245,12038.4,2832.105039,972.597777,22419.386178,1027.402223,0.34341868105128626,0.77


In [44]:
df['fuel_burn'].sum()

8914.096718758105